In [6]:
#Import library
import requests
from bs4 import BeautifulSoup
import io
from PyPDF2 import PdfFileReader
import numpy as np


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [14]:
# Setup WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Operasikan di background
service = Service('F:/proyek/Banking Credit Analysis/chromedriver.exe')  # Path ke chromedriver
driver = webdriver.Chrome(service=service, options=chrome_options)

# link ke halaman laporan per bulan
def get_report_links(url, keyword, start_year, end_year):
    report_links = []
    driver.get(url)

    while True:
        time.sleep(60)
        # Paginasi
        try:
            # Ambil semua link laporan tahunan
            a_tags = driver.find_elements(By.XPATH, "//a[contains(text(), '{}')]".format(keyword))
            for a_tag in a_tags:
                link_text = a_tag.text.strip()
                for year in range(start_year, end_year + 1):
                    if str(year) in link_text:
                        href = a_tag.get_attribute('href')
                        if href:
                            report_links.append(href)

            last_disabled_button = driver.find_element(By.XPATH, "//span[@class='disabledbluebutton' and contains(text(), '10')]")
            if last_disabled_button:
                ellipsis_button = driver.find_element(By.LINK_TEXT, '...')
                ellipsis_button.click()
            else: 
                for page_num in range(1, 11):
                    next_button = driver.find_element(By.XPATH, f"//a[@class='bluebutton' and contains(text(), {page_num+1})]")
                next_button.click()
        
        except:
            break

    return report_links

# get link PDF dari halaman laporan bulanan
def get_pdf_link(report_url):
    pdf_links = []
    driver.get(report_url)
    time.sleep(60)

    a_tags = driver.find_elements(By.XPATH, "//a[contains(@href, '.pdf')]")
    for a_tag in a_tags:
        pdf_link = a_tag.get_attribute('href')
        if pdf_link:
            pdf_links.append(pdf_link)

    return pdf_links

In [15]:
url_main = "https://ojk.go.id/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/default.aspx"
keyword = "Statistik Perbankan Indonesia"
start_year = 2003
end_year = 2023

# jalanin functionnya
report_links = get_report_links(url_main, keyword, start_year, end_year)

#concat dengan link pdf
pdf_links = []
for report_link in report_links:
    pdf_links.extend(get_pdf_link(report_link))

for pdf_link in pdf_links:
    print(pdf_link)

# Tutup WebDriver
driver.quit()

In [11]:
#save ke csv
import csv

def save_to_csv(pdf_links, filename='all_report_links.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        writer.writerow(['Statistik Perbankan OJK 2003-2023'])
        
        for link in pdf_links:
            writer.writerow([link])
    
    print(f'All links saved to {filename}')

save_to_csv(pdf_links)

All links saved to all_report_links.csv


In [5]:
#unduh pdf dan baca halaman tertentu dari pdf